In [1]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import plotly.graph_objs as go
from dash.dependencies import Input, Output
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas_datareader as web


app = JupyterDash(__name__)
server = app.server

scaler=MinMaxScaler(feature_range=(0,1))

#Pull data
company_tickers = 'AMZN'
start_date = '2015-10-31'
end_date = '2020-10-31'

df = web.DataReader(company_tickers, 'yahoo', start_date, end_date)
df.reset_index(inplace=True)

df["Date"]=pd.to_datetime(df.Date,format="%Y-%m-%d")
df.index=df['Date']


data=df.sort_index(ascending=True,axis=0)
new_data=pd.DataFrame(index=range(0,len(df)),columns=['Date','Close'])

for i in range(0,len(data)):
    new_data["Date"][i]=data['Date'][i]
    new_data["Close"][i]=data["Close"][i]

new_data.index=new_data.Date
new_data.drop("Date",axis=1,inplace=True)

dataset=new_data.values

training_data_len = int(np.ceil(len(df) * .8 ))
train=dataset[0:training_data_len,:]
valid=dataset[training_data_len:,:]

scaler=MinMaxScaler(feature_range=(0,1))
scaled_data=scaler.fit_transform(dataset)

x_train,y_train=[],[]

for i in range(60,len(train)):
    x_train.append(scaled_data[i-60:i,0])
    y_train.append(scaled_data[i,0])
    
x_train,y_train=np.array(x_train),np.array(y_train)

x_train=np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))

model=load_model("saved_model.h5")

inputs=new_data[len(new_data)-len(valid)-60:].values
inputs=inputs.reshape(-1,1)
inputs=scaler.transform(inputs)

X_test=[]
for i in range(60,inputs.shape[0]):
    X_test.append(inputs[i-60:i,0])
X_test=np.array(X_test)

X_test=np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))
closing_price=model.predict(X_test)
closing_price=scaler.inverse_transform(closing_price)

train=new_data[:training_data_len]
valid=new_data[training_data_len:]
valid['Predictions']=closing_price


if __name__=='__main__':
    app.run_server(debug=True)

<ipython-input-1-dc7d833941c5>:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid['Predictions']=closing_price


OSError: Address 'http://127.0.0.1:8050' already in use.
    Try passing a different port to run_server.